In [1]:
import pandas as pd
from statsmodels.stats.weightstats import ttest_ind

df = pd.read_parquet('data/hh.parquet')
services = pd.read_parquet('data/services.parquet')
food = pd.read_parquet('data/food.parquet')
health = pd.read_parquet('data/health.parquet')
edu = pd.read_parquet('data/education.parquet')

---

### Elegibility - everyone; treatment - used pmgky

In [2]:
df['treat'] = df['has_benefited_from_pmgky'] == 'yes'

ix = df.index
for i in [food, health, edu]:
    ix = i.index.intersection(ix)
df = df.loc[ix]

df['FOOD'] = food.loc[df.index].sum(axis=1)
df['EDUCATION'] = edu.loc[df.index].sum(axis=1)
df['HEALTH'] = health.loc[df.index].sum(axis=1)
df.head()

,sector,nss_region,multiplier,family_size,n_children,n_schoolchildren,n_elderly,n_school_meals,employed_annual,nco_2015,...,hoh_religion,caste,energy_source_cooking,energy_source_lighting,rationcard_type,has_benefited_from_pmgky,treat,FOOD,EDUCATION,HEALTH
hhid,,,,,,,,,,,,,,,,,,,,,
22300302,urban,341,28599,5,0,0,2,0.0,yes,112.0,...,Hinduism,other backward class (OBC),LPG,electricity (incl. generated by solar or wind ...,Above Poverty Line (APL),no,False,19071.666667,550.000000,133.333333
22300305,urban,341,28599,8,2,2,2,0.0,yes,112.0,...,Jainism,other backward class (OBC),LPG,electricity (incl. generated by solar or wind ...,Above Poverty Line (APL),no,False,27221.000000,175.000000,241.666667
22300312,urban,341,28599,6,2,0,2,0.0,yes,112.0,...,Hinduism,other backward class (OBC),LPG,electricity (incl. generated by solar or wind ...,Above Poverty Line (APL),no,False,21395.000000,316.666667,141.666667
22300313,urban,341,28599,3,1,1,0,0.0,yes,143.0,...,Hinduism,other backward class (OBC),LPG,electricity (incl. generated by solar or wind ...,Above Poverty Line (APL),no,False,15562.666667,233.333333,4000.000000
22301301,urban,341,16099,4,1,2,0,22.0,yes,112.0,...,Hinduism,other backward class (OBC),LPG,electricity (incl. generated by solar or wind ...,Below Poverty Line (BPL),no,False,9921.000000,225.000000,70.833333


In [3]:
deltas = df.groupby(['sector', 'treat']).apply(lambda x: (x[['FOOD', 'EDUCATION', 'HEALTH']] * x['multiplier'].values.reshape(-1, 1)).sum(axis=0) / x['multiplier'].sum())
deltas

FOOD    EDUCATION      HEALTH
sector treat                                       
rural  False  10349.025964   932.757792  366.254742
       True    9569.603057   808.078353  344.628828
urban  False  12237.092021  1246.073617  416.460897
       True   10815.570909  1153.424000  417.701387

In [4]:
def ttest(x, cols, alternative='two-sided'):
    trix = x[x['treat']].index
    crix = x[~x['treat']].index
    pvals = {}
    for col in cols:
        _, p, _ = ttest_ind(x.loc[crix, col], x.loc[trix, col], weights=(x.loc[crix, 'multiplier'], x.loc[trix, 'multiplier']), alternative=alternative)
        pvals[col] = p
    return pd.Series(pvals)

df.groupby('sector').apply(lambda x: ttest(x, ['FOOD', 'EDUCATION', 'HEALTH']))

,FOOD,EDUCATION,HEALTH
sector,,,
rural,0.0,0.0,0.0
urban,0.0,0.0,0.0


In [5]:
edf = edu.loc[df.index]

In [6]:
edf['sector'] = df['sector']
edf['multiplier'] = df['multiplier']
edf['treat'] = df['treat']
edf.head()

,books_1st,books_2nd,stationery,fees,coaching,edu_other,sector,multiplier,treat
hhid,,,,,,,,,
22300302,375.000000,NaN,66.666667,NaN,NaN,108.333333,urban,28599,False
22300305,NaN,NaN,150.000000,25.000000,NaN,NaN,urban,28599,False
22300312,208.333333,NaN,41.666667,NaN,NaN,66.666667,urban,28599,False
22300313,125.000000,NaN,41.666667,NaN,NaN,66.666667,urban,28599,False
22301301,NaN,NaN,41.666667,183.333333,NaN,NaN,urban,16099,False


In [7]:
cols = edf.drop(['treat', 'sector', 'multiplier'], axis=1).columns
edf.groupby(['sector', 'treat']).apply(lambda x: (x[cols] * x['multiplier'].values.reshape(-1, 1)).sum(axis=0) / x['multiplier'].sum())

books_1st  books_2nd  stationery        fees    coaching  \
sector treat                                                              
rural  False  161.301252   6.679362  101.837223  467.108165  170.779421   
       True   144.358607   4.739232   98.459299  381.472349  158.832753   
urban  False  244.681140   8.925427  126.516306  582.655828  230.881308   
       True   225.166049   6.418330  112.837920  579.187019  196.318569   

              edu_other  
sector treat             
rural  False  25.052369  
       True   20.216113  
urban  False  52.413608  
       True   33.496113

In [8]:
edf.fillna(0).groupby('sector').apply(lambda x: ttest(x, cols))

,books_1st,books_2nd,stationery,fees,coaching,edu_other
sector,,,,,,
rural,0.0,0.0,0.0,0.0,0.0,0.0
urban,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
fdf = food.loc[df.index]
fdf['sector'] = df['sector']
fdf['multiplier'] = df['multiplier']
fdf['treat'] = df['treat']
fdf.head()

,cereals,pulses,sugar_salt,dairy,veg,fruits,meat,oil,spice,bev,cooked,processed,sector,multiplier,treat
hhid,,,,,,,,,,,,,,,
22300302,2100.0,921.0,156.0,2643.333333,1486.333333,2816.666667,2513.333333,975.000000,1200.333333,1573.000000,1473.333333,1213.333333,urban,28599,False
22300305,2335.0,1075.0,320.0,3640.000000,2747.333333,1885.000000,4983.333333,3033.333333,2357.333333,901.333333,1646.666667,2296.666667,urban,28599,False
22300312,2235.0,970.0,172.0,2556.666667,2201.333333,1841.666667,5070.000000,1343.333333,1178.666667,1529.666667,1126.666667,1170.000000,urban,28599,False
22300313,1455.0,689.0,163.0,1906.666667,1495.000000,1863.333333,2808.000000,1161.333333,897.000000,936.000000,1083.333333,1105.000000,urban,28599,False
22301301,1515.0,458.0,122.0,888.333333,1105.000000,1040.000000,1863.333333,693.333333,641.333333,641.333333,173.333333,780.000000,urban,16099,False


In [10]:
cols = fdf.drop(['treat', 'sector', 'multiplier'], axis=1).columns
fdf.groupby(['sector', 'treat']).apply(lambda x: (x[cols] * x['multiplier'].values.reshape(-1, 1)).sum(axis=0) / x['multiplier'].sum())

cereals      pulses  sugar_salt        dairy          veg  \
sector treat                                                                  
rural  False  1212.387734  415.487745  192.635879  2128.393116  1239.963128   
       True   1020.127459  430.858926  185.685543  1705.848898  1256.254446   
urban  False  1231.445539  420.043785  179.375909  2539.603242  1235.541078   
       True   1060.829427  443.399415  174.031926  1920.672964  1251.195418   

                  fruits         meat         oil       spice         bev  \
sector treat                                                                
rural  False  614.357853  1023.133066  578.568926  673.646713  560.807858   
       True   565.965508  1035.426959  595.211212  666.174194  477.697959   
urban  False  852.834877   968.882938  559.940160  670.860528  858.099841   
       True   737.283775  1247.466132  563.390433  699.377197  688.928776   

                   cooked    processed  
sector treat                            
rural  False   973.895289   735.748657  
       True    949.154814   681.197140  
urban  False  1620.076129  1100.387996  
       True   1178.139654   850.855792

In [11]:
fdf.fillna(0).groupby('sector').apply(lambda x: ttest(x, cols))

,cereals,pulses,sugar_salt,dairy,veg,fruits,meat,oil,spice,bev,cooked,processed
sector,,,,,,,,,,,,
rural,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
urban,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
hdf = health.loc[df.index]
hdf['sector'] = df['sector']
hdf['multiplier'] = df['multiplier']
hdf['treat'] = df['treat']
hdf.head()

item_code,410,411,412,413,414,420,421,422,423,424,sector,multiplier,treat
hhid,,,,,,,,,,,,,
22300302,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,33.333333,NaN,NaN,urban,28599,False
22300305,NaN,NaN,NaN,NaN,NaN,208.333333,NaN,33.333333,NaN,NaN,urban,28599,False
22300312,NaN,NaN,NaN,NaN,NaN,108.333333,NaN,33.333333,NaN,NaN,urban,28599,False
22300313,1250.0,375.0,2083.333333,208.333333,NaN,66.666667,NaN,16.666667,NaN,NaN,urban,28599,False
22301301,NaN,NaN,NaN,NaN,NaN,54.166667,NaN,16.666667,NaN,NaN,urban,16099,False


In [13]:
cols = hdf.drop(['treat', 'sector', 'multiplier'], axis=1).columns
hdf.groupby(['sector', 'treat']).apply(lambda x: (x[cols] * x['multiplier'].values.reshape(-1, 1)).sum(axis=0) / x['multiplier'].sum())

item_code            410        411        412        413        414  \
sector treat                                                           
rural  False   94.334294  50.416197  48.437937  66.727730  23.324409   
       True    95.629450  47.271735  46.556804  61.028146  20.019237   
urban  False   95.625868  58.467749  62.249681  80.814900  22.706453   
       True   108.213684  58.905953  60.415810  79.104993  25.268222   

item_code           420       421        422       423       424  
sector treat                                                      
rural  False  63.033844  7.230027  10.355794  0.472141  1.922368  
       True   56.237955  6.420207   9.116025  0.360035  1.989234  
urban  False  70.904087  7.767489  14.686008  0.518483  2.720177  
       True   63.423681  7.996048  12.039996  0.594231  1.738770

In [14]:
hdf.fillna(0).groupby('sector').apply(lambda x: ttest(x, cols))

,410,411,412,413,414,420,421,422,423,424
sector,,,,,,,,,,
rural,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
urban,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

### Treatment = used ration card; eligibility, everyone

In [15]:
ration = pd.read_parquet('data/ration.parquet')

In [16]:
df = pd.read_parquet('data/hh.parquet')
services = pd.read_parquet('data/services.parquet')
food = pd.read_parquet('data/food.parquet')
health = pd.read_parquet('data/health.parquet')
edu = pd.read_parquet('data/education.parquet')

ix = df.index.intersection(ration.index)
df, ration = df.loc[ix], ration.loc[ix]

df['treat'] = ration['used_ration_card'] == 'yes'

ix = df.index
for i in [food, health, edu]:
    ix = i.index.intersection(ix)
df = df.loc[ix]

df['FOOD'] = food.loc[df.index].sum(axis=1)
df['EDUCATION'] = edu.loc[df.index].sum(axis=1)
df['HEALTH'] = health.loc[df.index].sum(axis=1)

In [17]:
deltas_ration = df.groupby(['sector', 'treat']).apply(lambda x: (x[['FOOD', 'EDUCATION', 'HEALTH']] * x['multiplier'].values.reshape(-1, 1)).sum(axis=0) / x['multiplier'].sum())
deltas_ration

FOOD    EDUCATION      HEALTH
sector treat                                       
rural  False  10900.521936  1063.634969  410.798483
       True    9623.916368   808.541931  340.497859
urban  False  12389.907199  1289.533123  399.384381
       True   11101.006979  1137.748141  434.789959

In [18]:
df.fillna(0).groupby('sector').apply(lambda x: ttest(x, ['FOOD', 'EDUCATION', 'HEALTH']))

,FOOD,EDUCATION,HEALTH
sector,,,
rural,0.0,0.0,0.0
urban,0.0,0.0,0.0


In [19]:
edf = edu.loc[df.index]
edf['sector'] = df['sector']
edf['multiplier'] = df['multiplier']
edf['treat'] = df['treat']
edf.head()

,books_1st,books_2nd,stationery,fees,coaching,edu_other,sector,multiplier,treat
hhid,,,,,,,,,
22300302,375.000000,NaN,66.666667,NaN,NaN,108.333333,urban,28599,False
22300305,NaN,NaN,150.000000,25.000000,NaN,NaN,urban,28599,False
22300312,208.333333,NaN,41.666667,NaN,NaN,66.666667,urban,28599,False
22300313,125.000000,NaN,41.666667,NaN,NaN,66.666667,urban,28599,False
22301301,NaN,NaN,41.666667,183.333333,NaN,NaN,urban,16099,False


In [20]:
cols = edf.drop(['treat', 'sector', 'multiplier'], axis=1).columns
edf.groupby(['sector', 'treat']).apply(lambda x: (x[cols] * x['multiplier'].values.reshape(-1, 1)).sum(axis=0) / x['multiplier'].sum())

books_1st  books_2nd  stationery        fees    coaching  \
sector treat                                                              
rural  False  189.267658   5.838181  117.878307  553.086034  166.517162   
       True   142.458560   5.301287   96.062728  382.545555  162.116814   
urban  False  261.525240   8.409188  136.680816  588.151528  237.025161   
       True   213.995057   7.725796  106.687993  574.641468  200.773778   

              edu_other  
sector treat             
rural  False  31.047628  
       True   20.056986  
urban  False  57.741191  
       True   33.924050

In [21]:
edf.fillna(0).groupby('sector').apply(lambda x: ttest(x, cols))

,books_1st,books_2nd,stationery,fees,coaching,edu_other
sector,,,,,,
rural,0.0,0.0,0.0,0.0,0.0,0.0
urban,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
fdf = food.loc[df.index]
fdf['sector'] = df['sector']
fdf['multiplier'] = df['multiplier']
fdf['treat'] = df['treat']
fdf.head()

,cereals,pulses,sugar_salt,dairy,veg,fruits,meat,oil,spice,bev,cooked,processed,sector,multiplier,treat
hhid,,,,,,,,,,,,,,,
22300302,2100.0,921.0,156.0,2643.333333,1486.333333,2816.666667,2513.333333,975.000000,1200.333333,1573.000000,1473.333333,1213.333333,urban,28599,False
22300305,2335.0,1075.0,320.0,3640.000000,2747.333333,1885.000000,4983.333333,3033.333333,2357.333333,901.333333,1646.666667,2296.666667,urban,28599,False
22300312,2235.0,970.0,172.0,2556.666667,2201.333333,1841.666667,5070.000000,1343.333333,1178.666667,1529.666667,1126.666667,1170.000000,urban,28599,False
22300313,1455.0,689.0,163.0,1906.666667,1495.000000,1863.333333,2808.000000,1161.333333,897.000000,936.000000,1083.333333,1105.000000,urban,28599,False
22301301,1515.0,458.0,122.0,888.333333,1105.000000,1040.000000,1863.333333,693.333333,641.333333,641.333333,173.333333,780.000000,urban,16099,False


In [23]:
cols = fdf.drop(['treat', 'sector', 'multiplier'], axis=1).columns
fdf.groupby(['sector', 'treat']).apply(lambda x: (x[cols] * x['multiplier'].values.reshape(-1, 1)).sum(axis=0) / x['multiplier'].sum())

cereals      pulses  sugar_salt        dairy          veg  \
sector treat                                                                  
rural  False  1488.583702  427.785081  216.485346  2560.837503  1243.059810   
       True   1006.501848  425.319859  182.519782  1709.560927  1252.299654   
urban  False  1317.127664  409.945201  181.224444  2689.801837  1211.306473   
       True   1026.179875  446.474191  173.801262  1959.340991  1271.128914   

                  fruits         meat         oil       spice         bev  \
sector treat                                                                
rural  False  633.260896   764.844603  606.232595  669.362140  547.056612   
       True   572.294888  1082.692107  586.447475  668.540408  497.478710   
urban  False  853.411942   773.841359  556.790000  649.541255  882.857599   
       True   772.606603  1360.465199  565.541594  712.329865  716.173004   

                   cooked    processed  
sector treat                            
rural  False   995.933718   747.079931  
       True    950.004884   690.255828  
urban  False  1747.550342  1116.509082  
       True   1185.051424   911.914057

In [24]:
fdf.fillna(0).groupby('sector').apply(lambda x: ttest(x, cols))

,cereals,pulses,sugar_salt,dairy,veg,fruits,meat,oil,spice,bev,cooked,processed
sector,,,,,,,,,,,,
rural,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
urban,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
hdf = health.loc[df.index]
hdf['sector'] = df['sector']
hdf['multiplier'] = df['multiplier']
hdf['treat'] = df['treat']
hdf.head()

item_code,410,411,412,413,414,420,421,422,423,424,sector,multiplier,treat
hhid,,,,,,,,,,,,,
22300302,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,33.333333,NaN,NaN,urban,28599,False
22300305,NaN,NaN,NaN,NaN,NaN,208.333333,NaN,33.333333,NaN,NaN,urban,28599,False
22300312,NaN,NaN,NaN,NaN,NaN,108.333333,NaN,33.333333,NaN,NaN,urban,28599,False
22300313,1250.0,375.0,2083.333333,208.333333,NaN,66.666667,NaN,16.666667,NaN,NaN,urban,28599,False
22301301,NaN,NaN,NaN,NaN,NaN,54.166667,NaN,16.666667,NaN,NaN,urban,16099,False


In [26]:
cols = hdf.drop(['treat', 'sector', 'multiplier'], axis=1).columns
hdf.groupby(['sector', 'treat']).apply(lambda x: (x[cols] * x['multiplier'].values.reshape(-1, 1)).sum(axis=0) / x['multiplier'].sum())

item_code            410        411        412        413        414  \
sector treat                                                           
rural  False  108.994147  56.065303  54.903532  76.382529  26.133579   
       True    92.536035  46.830659  45.697925  60.341383  20.158835   
urban  False   90.342592  56.078093  59.093898  78.097670  22.663349   
       True   109.707675  61.215056  64.215039  82.416929  24.516131   

item_code           420       421        422       423       424  
sector treat                                                      
rural  False  66.847762  7.882804  10.736455  0.498320  2.354052  
       True   56.903292  6.461258   9.298156  0.378093  1.892222  
urban  False  68.583455  7.252932  13.924087  0.584320  2.763985  
       True   68.123259  8.451549  13.642039  0.503319  1.998963

In [27]:
hdf.fillna(0).groupby('sector').apply(lambda x: ttest(x, cols))

,410,411,412,413,414,420,421,422,423,424
sector,,,,,,,,,,
rural,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
urban,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
